# DFx ETL Pipeline

## unstats.un.org

An ETL pipeline for [UNSD SDG API](https://unstats.un.org/SDGAPI/swagger/) from the United Nations Statistics Division (UNSD).

### Libraries

In [ ]:
import pandas as pd
from tqdm import tqdm

from dfpp.sources import unstats_un_org as source
from dfpp.storage import LocalStorage as Storage

storage = Storage()

### Extract

In [ ]:
df_metadata = source.get_series_metadata()
print("Shape:", df_metadata.shape)
display(df_metadata.head())

In [ ]:
data = []
for series_id in tqdm(df_metadata["code"].sample(5)):
    try:
        df = source.get_series_data(series_id)
    except Exception as error:
        print(series_id, error)
        continue
    data.append(df)
df_raw = pd.concat(data, axis=0, ignore_index=True)
print("Shape:", df_raw.shape)
display(df_raw.head())

### Transform

In [ ]:
df_transformed = source.transform(df_raw)
print("Shape:", df_transformed.shape)
display(df_transformed.head())

### Load

In [ ]:
for series_id, df in tqdm(df_transformed.groupby("series_id")):
    df.name = series_id
    storage.publish_dataset(df, folder_path=source.SOURCE_NAME)